In [ ]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
import os
from datetime import datetime, timedelta
import logging
import time
import json

APIKEY = "" # Replace with your actual API key

In [212]:
# Read CSV file
df = pd.read_csv('station66_distinct_clean.csv')
df2 = pd.read_csv('thailand_province_amphoe_tambon.csv')
    
# Clean THAI NUMBER in location
thai_numbers = str.maketrans("๐๑๒๓๔๕๖๗๘๙", "0123456789")
df["location"] = df["location"].str.translate(thai_numbers)
# Clean texts in location
keywords = ["เต็นท์", "เต้นท์", "ปะรำ", "บริเวณ"]
for text in keywords:
    df["location"] = df["location"].str.replace(text, "", regex=False).str.strip()
# Clean bracketed numbers in location
df["location"] = df["location"].str.replace(r"\(\d+\)", " ", regex=True).str.strip()
# Clean bracketed single characters in location
df["location"] = df["location"].str.replace(r"\([ก-ฮ]\)", " ", regex=True).str.strip()
# Clean double spaces in location
df["location"] = df["location"].str.replace(r"\s+", " ", regex=True).str.strip()
# Clean space before/after parenthese in location
df["location"] = df["location"].str.replace(r"\s+\(", "(", regex=True).str.replace(r"\)\s+", ")", regex=True).str.strip()
# Add space before/after parenthese in location
df["location"] = df["location"].str.replace(r"\(", " (", regex=True).str.replace(r"\)", ") ", regex=True).str.strip()
# CLean space before/after hyphen in location
df["location"] = df["location"].str.replace(r"\s*-\s*", "-", regex=True).str.strip()
# Clean spaces after text
df["location"] = df["location"].str.replace(r"\s+", " ", regex=True).str.strip()

# Save cleaned df to new CSV
df.to_csv('station66_distinct_clean_super.csv', index=False)

# Clean df.registrar text in all rows
keywords = ["ท้องถิ่น", "เทศบาล", "อำเภอ", "ตำบล", "เขต", "แขวง"]
df["proposed_district"] = df["registrar"]
for text in keywords:
    df["proposed_district"] = df["proposed_district"].str.replace(text, "", regex=False).str.strip()

display(df, df2.head())

,provinceNumber,province,registrar_code,registrar,subdis_code,subdistrict,electorate,location,latitude,longitude,proposed_district
0,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,หอประชุม มหาวิทยาลัยศิลปากร ถนนมหาราช,13.752802,100.489581,พระนคร
1,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,โรงเรียนวัดมหาธาตุ ถนนพระจันทร์,13.755687,100.490966,พระนคร
2,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,ศาลาหน้าอาคารวาสนะประทานวิทยาสิทธิ์ 1 โรงเรียน...,13.745237,100.495748,พระนคร
3,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,โรงเรียนวัดพระเชตุพน ถนนมหาราช,13.745582,100.491293,พระนคร
4,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100102,วังบูรพาภิรมย์,1,ตึกสามัคยาจารย์ โรงเรียนสวนกุหลาบวิทยาลัย ถนนต...,13.743462,100.498851,พระนคร
...,...,...,...,...,...,...,...,...,...,...,...
81422,96,นราธิวาส,9699,ท้องถิ่นเทศบาลเมืองนราธิวาส,960101,บางนาค,1,หอประชุมวิทยาลัยเทคนิคนราธิวาส,NaN,NaN,เมืองนราธิวาส
81423,96,นราธิวาส,9699,ท้องถิ่นเทศบาลเมืองนราธิวาส,960101,บางนาค,1,อาคารอเนกประสงค์โรงเรียนเทศบาล 5,NaN,NaN,เมืองนราธิวาส
81424,96,นราธิวาส,9699,ท้องถิ่นเทศบาลเมืองนราธิวาส,960101,บางนาค,1,ชั้นล่างอาคารเรียน 2 โรงเรียนเทศบาล 5,NaN,NaN,เมืองนราธิวาส
81425,96,นราธิวาส,9699,ท้องถิ่นเทศบาลเมืองนราธิวาส,960101,บางนาค,1,ศาลาวัดประชาภิรมย์ 1,NaN,NaN,เมืองนราธิวาส


,ADM3_EN,ADM3_TH,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE
0,Phraborom Maharatchawang,พระบรมมหาราชวัง,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
1,Wang Burapha Phirom,วังบูรพาภิรมย์,TH100102,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
2,Wat Ratchabophit,วัดราชบพิธ,TH100103,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
3,Samran Rat,สำราญราษฎร์,TH100104,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
4,San Chaopho Suea,ศาลเจ้าพ่อเสือ,TH100105,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH


In [217]:
# Join df by df2 on subdistrict proposed_district and province
# !! Some subdistrict and province has multiple matches, causing duplicated rows !!
df_merged = pd.merge(df, df2, left_on=['subdistrict', 'proposed_district', 'province'], right_on=['ADM3_TH', 'ADM2_TH', 'ADM1_TH'], how='left')

# If missing ADM3_EN, join by subdistrict and province on the missing rows
df_merged_v2 = pd.merge(df, df2, left_on=['subdistrict', 'province'], right_on=['ADM3_TH', 'ADM1_TH'], how='left')
df_merged.update(df_merged_v2)
display(df_merged)
display(df_merged[df_merged['ADM3_EN'].isna()])

,provinceNumber,province,registrar_code,registrar,subdis_code,subdistrict,electorate,location,latitude,longitude,...,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE
0,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,หอประชุม มหาวิทยาลัยศิลปากร ถนนมหาราช,13.752802,100.489581,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
1,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,โรงเรียนวัดมหาธาตุ ถนนพระจันทร์,13.755687,100.490966,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
2,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,ศาลาหน้าอาคารวาสนะประทานวิทยาสิทธิ์ 1 โรงเรียน...,13.745237,100.495748,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
3,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,โรงเรียนวัดพระเชตุพน ถนนมหาราช,13.745582,100.491293,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
4,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100102,วังบูรพาภิรมย์,1,ตึกสามัคยาจารย์ โรงเรียนสวนกุหลาบวิทยาลัย ถนนต...,13.743462,100.498851,...,TH100102,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81422,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านลาหงา,NaN,NaN,...,TH910502,La-Ngu,ละงู,TH9105,Satun,สตูล,TH91,Thailand,ประเทศไทย,TH
81423,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนบ้านบากันโต๊ะทิด,NaN,NaN,...,TH910502,La-Ngu,ละงู,TH9105,Satun,สตูล,TH91,Thailand,ประเทศไทย,TH
81424,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนบ้านนาพญา,NaN,NaN,...,TH910502,La-Ngu,ละงู,TH9105,Satun,สตูล,TH91,Thailand,ประเทศไทย,TH
81425,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านคลองขุด,NaN,NaN,...,TH910502,La-Ngu,ละงู,TH9105,Satun,สตูล,TH91,Thailand,ประเทศไทย,TH


,provinceNumber,province,registrar_code,registrar,subdis_code,subdistrict,electorate,location,latitude,longitude,...,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE


In [ ]:
# Try one row
n = 12768
row = df_merged.iloc[n]
if row['province'] == "กรุงเทพมหานคร":
    address = f"{row['location']} แขวง{row['subdistrict']} เขต{row['ADM2_TH']} {row['province']} ประเทศไทย"
else:
    address = f"{row['location']} ตำบล{row['subdistrict']} อำเภอ{row['ADM2_TH']} จังหวัด{row['province']} ประเทศไทย"
print(address)

# Geocode function
def address_to_id(address, api_key=APIKEY):
    base_url = "https://places.googleapis.com/v1/places:searchText"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "places.id,places.name,nextPageToken",
    }
    body = {
        "textQuery": address,
        "languageCode": "th",
    }
    
    response = requests.post(base_url, headers=headers, json=body)
    if response.status_code == 200:
        result = response.json()
        if 'places' in result and len(result['places']) > 0:
            place_id = result['places'][0]['id']
            return place_id, result
        else:
            return None
    else:
        logging.error(f"Error {response.status_code}: {response.text}")
        return None
    
# Test the function
try:
    place_id, result = address_to_id(address)
except Exception as e:
    print(f"Error: {e}")
    place_id, result = None, None

ศาลาอเนกประสงค์บ้านบางอิน ตำบลคลองใหญ่ อำเภอแหลมงอบ จังหวัดตราด ประเทศไทย


In [215]:
display(place_id, result)
place_url = f"https://www.google.com/maps/place/?q=place_id:{place_id}"
print(place_url)

'ChIJp9Ov0-JDBDER7WRRYuCMYHY'

{'places': [{'name': 'places/ChIJp9Ov0-JDBDER7WRRYuCMYHY',
   'id': 'ChIJp9Ov0-JDBDER7WRRYuCMYHY'},
  {'name': 'places/ChIJRY8W6dhqBDERWRH8SBiaPSA',
   'id': 'ChIJRY8W6dhqBDERWRH8SBiaPSA'}]}

https://www.google.com/maps/place/?q=place_id:ChIJp9Ov0-JDBDER7WRRYuCMYHY


In [201]:
result["places"][0]["id"]

'ChIJD9qkNfoVGzERGsnfx8lAP5k'

In [202]:
# Run through all rows and save placeid_results
# Divide to 2 chunks
chunk_size = len(df_merged) // 2
testchunk = df_merged.iloc[0:5]
chunkn1 = df_merged.iloc[:chunk_size]
chunkn2 = df_merged.iloc[chunk_size:]
display(testchunk, chunkn1, chunkn2)

,provinceNumber,province,registrar_code,registrar,subdis_code,subdistrict,electorate,location,latitude,longitude,...,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE
0,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,หอประชุม มหาวิทยาลัยศิลปากร ถนนมหาราช,13.752802,100.489581,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
1,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,โรงเรียนวัดมหาธาตุ ถนนพระจันทร์,13.755687,100.490966,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
2,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,ศาลาหน้าอาคารวาสนะประทานวิทยาสิทธิ์ 1 โรงเรียน...,13.745237,100.495748,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
3,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,โรงเรียนวัดพระเชตุพน ถนนมหาราช,13.745582,100.491293,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
4,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100102,วังบูรพาภิรมย์,1,ตึกสามัคยาจารย์ โรงเรียนสวนกุหลาบวิทยาลัย ถนนต...,13.743462,100.498851,...,TH100102,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH


,provinceNumber,province,registrar_code,registrar,subdis_code,subdistrict,electorate,location,latitude,longitude,...,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE
0,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,หอประชุม มหาวิทยาลัยศิลปากร ถนนมหาราช,13.752802,100.489581,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
1,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,โรงเรียนวัดมหาธาตุ ถนนพระจันทร์,13.755687,100.490966,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
2,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,ศาลาหน้าอาคารวาสนะประทานวิทยาสิทธิ์ 1 โรงเรียน...,13.745237,100.495748,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
3,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,โรงเรียนวัดพระเชตุพน ถนนมหาราช,13.745582,100.491293,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
4,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100102,วังบูรพาภิรมย์,1,ตึกสามัคยาจารย์ โรงเรียนสวนกุหลาบวิทยาลัย ถนนต...,13.743462,100.498851,...,TH100102,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40708,41,อุดรธานี,4197,ท้องถิ่นเทศบาลตำบลนิคมสงเคราะห์,410119,โคกสะอาด,9,ศาลากลางบ้านนิคมฯ 2 หมู่ที่ 6 (หลังใหม่) ต.โคก...,NaN,NaN,...,TH410119,Mueang Udon Thani,เมืองอุดรธานี,TH4101,Udon Thani,อุดรธานี,TH41,Thailand,ประเทศไทย,TH
40709,41,อุดรธานี,4197,ท้องถิ่นเทศบาลตำบลนิคมสงเคราะห์,410119,โคกสะอาด,9,ศาลากลางบ้านนิคมฯ 3 (ที่่ทำการ อบต.โคกสะอาด หล...,NaN,NaN,...,TH410119,Mueang Udon Thani,เมืองอุดรธานี,TH4101,Udon Thani,อุดรธานี,TH41,Thailand,ประเทศไทย,TH
40710,41,อุดรธานี,4198,ท้องถิ่นเทศบาลตำบลนาข่า,410116,นาข่า,2,ศาลาวัดนาคาเทวี (นาข่า),NaN,NaN,...,TH410116,Mueang Udon Thani,เมืองอุดรธานี,TH4101,Udon Thani,อุดรธานี,TH41,Thailand,ประเทศไทย,TH
40711,41,อุดรธานี,4198,ท้องถิ่นเทศบาลตำบลนาข่า,410116,นาข่า,2,ศาลากลางบ้านนาข่า,NaN,NaN,...,TH410116,Mueang Udon Thani,เมืองอุดรธานี,TH4101,Udon Thani,อุดรธานี,TH41,Thailand,ประเทศไทย,TH


,provinceNumber,province,registrar_code,registrar,subdis_code,subdistrict,electorate,location,latitude,longitude,...,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE
40713,41,อุดรธานี,4198,ท้องถิ่นเทศบาลตำบลนาข่า,410116,นาข่า,2,ศาลาวัดโคกศรีสำราญ (ถ่อนใหญ่),NaN,NaN,...,TH410116,Mueang Udon Thani,เมืองอุดรธานี,TH4101,Udon Thani,อุดรธานี,TH41,Thailand,ประเทศไทย,TH
40714,41,อุดรธานี,4198,ท้องถิ่นเทศบาลตำบลนาข่า,410116,นาข่า,2,ศาลากลางบ้านถ่อนใหญ่,NaN,NaN,...,TH410116,Mueang Udon Thani,เมืองอุดรธานี,TH4101,Udon Thani,อุดรธานี,TH41,Thailand,ประเทศไทย,TH
40715,41,อุดรธานี,4198,ท้องถิ่นเทศบาลตำบลนาข่า,410116,นาข่า,2,สวนสาธารณะหนองภาโรน (ดอนแตง),NaN,NaN,...,TH410116,Mueang Udon Thani,เมืองอุดรธานี,TH4101,Udon Thani,อุดรธานี,TH41,Thailand,ประเทศไทย,TH
40716,41,อุดรธานี,4198,ท้องถิ่นเทศบาลตำบลนาข่า,410116,นาข่า,2,ศาลาวัดอัมพวัน (ดงยวด),NaN,NaN,...,TH410116,Mueang Udon Thani,เมืองอุดรธานี,TH4101,Udon Thani,อุดรธานี,TH41,Thailand,ประเทศไทย,TH
40717,41,อุดรธานี,4198,ท้องถิ่นเทศบาลตำบลนาข่า,410116,นาข่า,2,ศาลากลางบ้านดงยวด,NaN,NaN,...,TH410116,Mueang Udon Thani,เมืองอุดรธานี,TH4101,Udon Thani,อุดรธานี,TH41,Thailand,ประเทศไทย,TH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81422,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านลาหงา,NaN,NaN,...,TH910502,La-Ngu,ละงู,TH9105,Satun,สตูล,TH91,Thailand,ประเทศไทย,TH
81423,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนบ้านบากันโต๊ะทิด,NaN,NaN,...,TH910502,La-Ngu,ละงู,TH9105,Satun,สตูล,TH91,Thailand,ประเทศไทย,TH
81424,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนบ้านนาพญา,NaN,NaN,...,TH910502,La-Ngu,ละงู,TH9105,Satun,สตูล,TH91,Thailand,ประเทศไทย,TH
81425,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านคลองขุด,NaN,NaN,...,TH910502,La-Ngu,ละงู,TH9105,Satun,สตูล,TH91,Thailand,ประเทศไทย,TH


In [216]:
# Run through all rows in chunk
chunk = chunkn1
chunk_name = "chunk_1"
placeid_results = []
rate_limit_per_minute = 600  # Adjust as needed
from tqdm import tqdm
for index, row in tqdm(chunk.iterrows(), total=chunk.shape[0]):
    if row['province'] == "กรุงเทพมหานคร":
        address = f"{row['location']} แขวง{row['subdistrict']} เขต{row['ADM2_TH']} {row['province']} ประเทศไทย"
    else:
        address = f"{row['location']} ตำบล{row['subdistrict']} อำเภอ{row['ADM2_TH']} จังหวัด{row['province']} ประเทศไทย"
    try:
        place_id, result = address_to_id(address)
        placeid_results.append({
            'index': index,
            'location': row['location'],
            'address': address,
            'place_id': place_id,
            'place_url': f"https://www.google.com/maps/place/?q=place_id:{place_id}" if place_id else None,
            'result': result
        })
    except Exception as e:
        # logging.error(f"Error processing row {index} with address '{address}': {e}")
        placeid_results.append({
            'index': index,
            'location': row['location'],
            'address': address,
            'place_id': None,
            'place_url': None,
            'result': str(e)
        })
    # Sleep to respect rate limit
    # time.sleep(60.0 / rate_limit_per_minute)  # Adjust sleep time as needed to respect API rate limits
    
df_placeid = pd.DataFrame(placeid_results)
display(df_placeid)
# Save to CSV
df_placeid.to_csv(f'station66_placeid_{chunk_name}.csv', index=False)

 66%|██████▌   | 26713/40713 [3:42:17<1:52:30,  2.07it/s]ERROR:root:Error 503: {
  "error": {
    "code": 503,
    "message": "The service is currently unavailable.",
    "status": "UNAVAILABLE"
  }
}

100%|██████████| 40713/40713 [5:44:22<00:00,  1.97it/s]  


,index,location,address,place_id,place_url,result
0,0,หอประชุม มหาวิทยาลัยศิลปากร ถนนมหาราช,หอประชุม มหาวิทยาลัยศิลปากร ถนนมหาราช แขวงพระบ...,ChIJsQW2s6CZ4jARKCwV_40G50c,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJsQW2s6CZ4jARK...
1,1,โรงเรียนวัดมหาธาตุ ถนนพระจันทร์,โรงเรียนวัดมหาธาตุ ถนนพระจันทร์ แขวงพระบรมมหาร...,ChIJc3ySHQWZ4jAR8ocYUwGc9mM,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJc3ySHQWZ4jAR8...
2,2,ศาลาหน้าอาคารวาสนะประทานวิทยาสิทธิ์ 1 โรงเรียน...,ศาลาหน้าอาคารวาสนะประทานวิทยาสิทธิ์ 1 โรงเรียน...,ChIJV_-yxQSZ4jAR2GWdpHhQYVM,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJV_-yxQSZ4jAR2...
3,3,โรงเรียนวัดพระเชตุพน ถนนมหาราช,โรงเรียนวัดพระเชตุพน ถนนมหาราช แขวงพระบรมมหารา...,ChIJCRWKpOmZ4jARCM4KBhoWp6A,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJCRWKpOmZ4jARC...
4,4,ตึกสามัคยาจารย์ โรงเรียนสวนกุหลาบวิทยาลัย ถนนต...,ตึกสามัคยาจารย์ โรงเรียนสวนกุหลาบวิทยาลัย ถนนต...,ChIJaRiqxauZ4jARilfgXfsAobU,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJaRiqxauZ4jARi...
...,...,...,...,...,...,...
40708,40708,ศาลากลางบ้านนิคมฯ 2 หมู่ที่ 6 (หลังใหม่) ต.โคก...,ศาลากลางบ้านนิคมฯ 2 หมู่ที่ 6 (หลังใหม่) ต.โคก...,ChIJoftO4MWbIzERL687Kw_QYJU,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJoftO4MWbIzERL...
40709,40709,ศาลากลางบ้านนิคมฯ 3 (ที่่ทำการ อบต.โคกสะอาด หล...,ศาลากลางบ้านนิคมฯ 3 (ที่่ทำการ อบต.โคกสะอาด หล...,ChIJ-QqtE7iZIzERG8ORvBozIuk,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJ-QqtE7iZIzERG...
40710,40710,ศาลาวัดนาคาเทวี (นาข่า),ศาลาวัดนาคาเทวี (นาข่า) ตำบลนาข่า อำเภอเมืองอุ...,ChIJRejG1-OBIzERr91XHZFIGXw,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJRejG1-OBIzERr...
40711,40711,ศาลากลางบ้านนาข่า,ศาลากลางบ้านนาข่า ตำบลนาข่า อำเภอเมืองอุดรธานี...,ChIJz1nTcuGBIzER-K7CEeDHvWE,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJz1nTcuGBIzER-...


In [218]:
# Count successful and failed requests
success_count = df_placeid['place_id'].notna().sum()
failure_count = df_placeid['place_id'].isna().sum()
all_count = len(df_placeid)
print(f"Successful requests: {success_count} ({success_count/all_count:.2%})")
print(f"Failed requests: {failure_count} ({failure_count/all_count:.2%})")

Successful requests: 37923 (93.15%)
Failed requests: 2790 (6.85%)
